In [1]:
import os
import sys
from time import sleep
import json
import pandas as pd
import os
import gc
import numpy as np
from tqdm.notebook import tqdm
import pickle

from kaggle_environments import (
    evaluate, make, utils,
    get_episode_replay,
)

root = "../input/meta-kaggle"
santa_id = 24539

episode_agents = pd.read_csv(os.path.join(root, "EpisodeAgents.csv"))
agents = pd.read_csv(os.path.join(root, "Episodes.csv"))

Loading environment football failed: No module named 'gfootball'


In [2]:
#get agent - competition mapping
santa_agents = agents.loc[(agents['CompetitionId'] == santa_id)].reset_index(drop = True)
gc.collect()

#get episode of santa competition
episode_agents = episode_agents.merge(santa_agents, how = 'inner', left_on = 'EpisodeId', right_on = "Id")
gc.collect()

0

In [3]:
keep_col = [
    "EpisodeId", "Index", "Reward", "State", "SubmissionId", "InitialConfidence",
    "InitialScore", "UpdatedConfidence", "UpdatedScore", "Type", "CreateTime", "EndTime"
]

episode_agents = episode_agents[keep_col]

#discard validation episode
episode_agents = episode_agents[episode_agents['Type'] == 1].reset_index(drop = True)

#convert time column to dt
episode_agents['CreateTime'] = pd.to_datetime(episode_agents['CreateTime'].transform(lambda x: x[:16]))
episode_agents['EndTime'] = pd.to_datetime(episode_agents['EndTime'].transform(lambda x: x[:16]))

gc.collect()

0

In [4]:
del agents, santa_agents
gc.collect()

20

In [5]:
#get last episode for each submission (last score on leaderboard) and sort by score
last_submission_update = episode_agents.sort_values(['SubmissionId', 'EndTime'], ascending = False).groupby('SubmissionId').head(1)
last_submission_update = last_submission_update.sort_values('UpdatedScore', ascending = False)

In [6]:
gc.collect()

#treshold for 7 position on leaderboard
top_7_tresh = 1233
not_so_well_tresh = 1100

#get every agent with UpdatedScore >= top_7_tresh
top_agents = last_submission_update[last_submission_update['UpdatedScore'] >= top_7_tresh].SubmissionId.tolist()

In [7]:
#get pulled_episode a list with every pulled episode

#insert here path of input folder
path_folder = '../input/agent-scrape-everything-step-1'

if path_folder != '':
    
    with open(os.path.join(path_folder, 'winner_dic.pkl'), 'rb') as file:
        winner_dic = pickle.load(file)
        
    pulled_episodes = [int(x.replace('.json', '')) for x in winner_dic.keys()]
              

else:
    pulled_episodes = []
    winner_dic  = {}

In [8]:
episodes_to_pull = []
for sub_id in tqdm(top_agents):
        
    epid_list_submission = episode_agents.loc[episode_agents['SubmissionId'] == sub_id, 'EpisodeId'].tolist()

    #get list of episode which hasn't been loaded yet
    list_episodes = [epid for epid in epid_list_submission if epid not in pulled_episodes]
    episodes_to_pull += list_episodes
    
print('Number of remaining Episodes', len(episodes_to_pull))
    
del episodes_to_pull
gc.collect()


Number of remaining Episodes 23457


3

In [9]:
#max number of call
gc.collect()
MAXAPICALLS = 3600
DIR =  './'
time_w8 = 20
num_api_calls_today = 0
counter_per_minute = 0

for sub_id in tqdm(top_agents):
    print(f'Starting scraping: {sub_id}')
    
    if num_api_calls_today < MAXAPICALLS:
        
        epid_list_submission = episode_agents.loc[episode_agents['SubmissionId'] == sub_id, 'EpisodeId'].tolist()
        
        #get list of episode which hasn't been loaded yet
        list_episodes = [epid for epid in epid_list_submission if epid not in pulled_episodes]
        
        print(f'{len(list_episodes)} different episodes')
        
        for epid in list_episodes:
            
            #get reward for each agent of the episode
            reward = episode_agents[episode_agents['EpisodeId'] == epid]['Reward'].tolist()
            
            #get_sub_id
            sub_episode_list = episode_agents[episode_agents['EpisodeId'] == epid]['SubmissionId'].tolist()
            
            #get last score per sub
            score_per_sub = [last_submission_update.loc[last_submission_update['SubmissionId'] == x, 'UpdatedScore'].values[0] for x in sub_episode_list]
            
            #this episode was broken and one agent got no reward
            if any([x is None for x in reward]): 
                pulled_episodes += [epid]
                
                #next episode
                continue 
                
            #get index for each agent
            agent = episode_agents[episode_agents['EpisodeId'] == epid]['Index'].tolist()
            
            winner_score = max(reward)
            winner = []
            for i in range(2):
                if (score_per_sub[i] >= top_7_tresh):
                    winner += [i]
                
                if (score_per_sub[i] >= not_so_well_tresh) & (reward[i] == winner_score):
                    winner += [i]
            
            #save top agent and winner perspective (if it has >= not_so_well_tresh)
            winner = list(set(winner))
            
            try:
                #get the episode
                replay = get_episode_replay(epid)['result']['replay']
                pulled_episodes += [epid]
                
            except:
                sleep(time_w8)

            # save replay
            with open(DIR + '/{}.json'.format(epid), 'w') as f:
                f.write(replay)
            
            #save the winner
            winner_dic[f"{epid}.json"] = winner 
            
            num_api_calls_today += 1
            counter_per_minute += 1
            
            if counter_per_minute >= 60:
                sleep(time_w8)
                counter_per_minute = 0
    
with open('winner_dic.pkl', 'wb') as file:
    pickle.dump(winner_dic, file, protocol=pickle.HIGHEST_PROTOCOL)

Starting scraping: 19145375
0 different episodes
Starting scraping: 18923141
0 different episodes
Starting scraping: 19123171
0 different episodes
Starting scraping: 18981696
0 different episodes
Starting scraping: 18890359
0 different episodes
Starting scraping: 18775142
0 different episodes
Starting scraping: 19052323
0 different episodes
Starting scraping: 18771699
0 different episodes
Starting scraping: 18806907
0 different episodes
Starting scraping: 19035513
0 different episodes
Starting scraping: 18948314
0 different episodes
Starting scraping: 18771536
0 different episodes
Starting scraping: 18835876
0 different episodes
Starting scraping: 18771608
0 different episodes
Starting scraping: 19147560
0 different episodes
Starting scraping: 18994253
0 different episodes
Starting scraping: 18870298
0 different episodes
Starting scraping: 18755750
0 different episodes
Starting scraping: 18778644
0 different episodes
Starting scraping: 19083255
0 different episodes
Starting scraping: 1